In [3]:
import pandas as pd
import numpy as np
import TSAEDistance as TSAE

In [4]:
def cluster_points(X, mu):
    clusters  = {}
    for x in X:
        bestmukey = min([(i[0], np.linalg.norm(x-mu[i[0]])) \
                    for i in enumerate(mu)], key=lambda t:t[1])[0]
        try:
            clusters[bestmukey].append(x)
        except KeyError:
            clusters[bestmukey] = [x]
    return clusters
 
def reevaluate_centers(mu, clusters):
    newmu = []
    keys = sorted(clusters.keys())
    for k in keys:
        newmu.append(np.mean(clusters[k], axis = 0))
    return newmu
 
def has_converged(mu, oldmu):
    return (set([tuple(a) for a in mu]) == set([tuple(a) for a in oldmu]))
 
def find_centers(X, K):
    # Initialize to K random centers
    oldmu = random.sample(X, K)
    mu = random.sample(X, K)
    while not has_converged(mu, oldmu):
        oldmu = mu
        # Assign all points in X to clusters
        clusters = cluster_points(X, mu)
        # Reevaluate centers
        mu = reevaluate_centers(oldmu, clusters)
    return(mu, clusters)

In [32]:
clusters = np.array([1, 1, 2, 2, 0, 1])

In [33]:
data = np.array(data)
print data

[[12 10 87]
 [ 2 12 33]
 [68 31 32]
 [88 13 66]
 [79 40 89]
 [ 1 77 12]]


In [34]:
np.mean(data[clusters == 1], axis = 0)

array([  5.,  33.,  44.])

In [5]:
def cluster_centroids(data, clusters, k=None):
    """Return centroids of clusters in data.

    data is an array of observations with shape (A, B, ...).

    clusters is an array of integers of shape (A,) giving the index
    (from 0 to k-1) of the cluster to which each observation belongs.
    The clusters must all be non-empty.

    k is the number of clusters. If omitted, it is deduced from the
    values in the clusters array.

    The result is an array of shape (k, B, ...) containing the
    centroid of each cluster.

    >>> data = np.array([[12, 10, 87],
    ...                  [ 2, 12, 33],
    ...                  [68, 31, 32],
    ...                  [88, 13, 66],
    ...                  [79, 40, 89],
    ...                  [ 1, 77, 12]])
    >>> cluster_centroids(data, np.array([1, 1, 2, 2, 0, 1]))
    array([[ 79.,  40.,  89.],
           [  5.,  33.,  44.],
           [ 78.,  22.,  49.]])

    """
    if k is None:
        k = np.max(clusters) + 1
    result = np.empty(shape=(k,) + data.shape[1:])
    for i in range(k):
        np.mean(data[clusters == i], axis=0, out=result[i])
    return result


In [25]:
data = np.array([[12, 10, 87], [ 2, 12, 33],[68, 31, 32], [88, 13, 66],[79, 40, 89],[ 1, 77, 12]])

In [26]:
cluster_centroids(data, np.array([1, 1, 2, 2, 0, 1]))

[[  4.94065646e-324   4.94065646e-324   4.94065646e-324]
 [  9.88131292e-324   9.88131292e-324   9.88131292e-324]
 [  1.48219694e-323   1.48219694e-323   1.48219694e-323]] result
[[12 10 87]
 [ 2 12 33]
 [68 31 32]
 [88 13 66]
 [79 40 89]
 [ 1 77 12]] data
[[  7.90000000e+001   4.00000000e+001   8.90000000e+001]
 [  9.88131292e-324   9.88131292e-324   9.88131292e-324]
 [  1.48219694e-323   1.48219694e-323   1.48219694e-323]] resulti
[[12 10 87]
 [ 2 12 33]
 [68 31 32]
 [88 13 66]
 [79 40 89]
 [ 1 77 12]] data
[[  7.90000000e+001   4.00000000e+001   8.90000000e+001]
 [  5.00000000e+000   3.30000000e+001   4.40000000e+001]
 [  1.48219694e-323   1.48219694e-323   1.48219694e-323]] resulti
[[12 10 87]
 [ 2 12 33]
 [68 31 32]
 [88 13 66]
 [79 40 89]
 [ 1 77 12]] data
[[ 79.  40.  89.]
 [  5.  33.  44.]
 [ 78.  22.  49.]] resulti


array([[ 79.,  40.,  89.],
       [  5.,  33.,  44.],
       [ 78.,  22.,  49.]])

In [6]:
import scipy.spatial

def kmeans(data, k=None, centroids=None, steps=20):
    """Divide the observations in data into clusters using the k-means
    algorithm, and return an array of integers assigning each data
    point to one of the clusters.

    centroids, if supplied, must be an array giving the initial
    position of the centroids of each cluster.

    If centroids is omitted, the number k gives the number of clusters
    and the initial positions of the centroids are selected randomly
    from the data.

    The k-means algorithm adjusts the centroids iteratively for the
    given number of steps, or until no further progress can be made.

    >>> data = np.array([[12, 10, 87],
    ...                  [ 2, 12, 33],
    ...                  [68, 31, 32],
    ...                  [88, 13, 66],
    ...                  [79, 40, 89],
    ...                  [ 1, 77, 12]])
    >>> np.random.seed(73)
    >>> kmeans(data, k=3)
    array([1, 1, 2, 2, 0, 1])

    """
    if centroids is not None and k is not None:
        assert(k == len(centroids))
    elif centroids is not None:
        k = len(centroids)
    elif k is not None:
        # Forgy initialization method: choose k data points randomly.
        centroids = data[np.random.choice(np.arange(len(data)), k, False)]
        print centroids, 'centroids'
    else:
        raise RuntimeError("Need a value for k or centroids.")

    for _ in range(max(steps, 1)):
        # Squared distances between each point and each centroid.
        sqdists = scipy.spatial.distance.cdist(centroids, data, 'sqeuclidean')
        print sqdists, 'sqdists'

        # Index of the closest centroid to each data point.
        clusters = np.argmin(sqdists, axis=0)
        print clusters, 'clusters'

        new_centroids = cluster_centroids(data, clusters, k)
        print new_centroids, 'new_centroids'
        if np.array_equal(new_centroids, centroids):
            break

        centroids = new_centroids

    return clusters

In [9]:
data = np.array([[12, 10, 87], [ 2, 12, 33],[68, 31, 32], [88, 13, 66],[79, 40, 89],[ 1, 77, 12]])
print data

[[12 10 87]
 [ 2 12 33]
 [68 31 32]
 [88 13 66]
 [79 40 89]
 [ 1 77 12]]


In [8]:
kmeans(data,k=3)

[[68 31 32]
 [79 40 89]
 [ 1 77 12]] centroids
[[  6602.   4718.      0.   1880.   3451.   7005.]
 [  5393.   9849.   3451.   1339.      0.  13382.]
 [ 10235.   4667.   7005.  14581.  13382.      0.]] sqdists
[1 2 0 1 1 2] clusters
[[ 68.          31.          32.        ]
 [ 59.66666667  21.          80.66666667]
 [  1.5         44.5         22.5       ]] new_centroids
[[  6602.           4718.              0.           1880.           3451.
    7005.        ]
 [  2433.22222222   5678.55555556   2537.88888889   1081.88888889
     804.22222222  11292.88888889]
 [  5460.75         1166.75         4694.75        10366.75        10448.75
    1166.75      ]] sqdists
[1 2 0 1 1 2] clusters
[[ 68.          31.          32.        ]
 [ 59.66666667  21.          80.66666667]
 [  1.5         44.5         22.5       ]] new_centroids


array([1, 2, 0, 1, 1, 2])